# Questão 1

In [1]:
import numpy as np 
from numpy import genfromtxt
bc_data = genfromtxt('breastcancer.csv', delimiter=',')
bc_data.shape

(569, 31)

In [2]:
bc_data_X = bc_data[:, :-1]
bc_data_y = bc_data[:, [-1]]


#### K-fold

In [10]:
def compute_accuracies(y_pred, y_val):
    by_class_accuracies = []
    accuracies = []


    #print(y_pred,y_val)
    #print('/n --------------')

    if  y_val.shape[1] > 1:
         y_val = np.argmax(y_val, axis=1)    

    y_pred = y_pred.ravel()         
    y_val = y_val.ravel().astype(int)  

    classes = np.unique(y_val)
    y_val = y_val.ravel()

    for k in classes: 
        y_pred_k = y_pred[y_val == k]
        y_val_k = y_val[y_val == k]
        class_acc = np.mean(y_pred_k==y_val_k)
        by_class_accuracies.append(class_acc)
    #   print(y_pred_k,y_val_k)
    
    accuracy = np.mean(y_pred == y_val)
    accuracies.append(accuracy)

    return accuracies, by_class_accuracies

In [4]:
# 10 fold cross validation:
def k_fold_cross_validation(X, y, predicao, k=10):
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    general = []
    by_class = []
    fold_size = len(X) // k
    
    for i in range(k):

        start = i * fold_size
        end = (i + 1) * fold_size if i != k - 1 else len(X)  
        val_indices = indices[start:end]
        train_indices = np.concatenate((indices[:start], indices[end:]))

        X_treino, y_treino = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]

        mean = X_treino.mean(axis=0)
        std = X_treino.std(axis=0, ddof=1)

        std[std == 0] = 1
        X_treino_norm = (X_treino - mean) / std
        X_val_norm = (X_val - mean) / std

        y_pred = predicao(X_treino_norm, y_treino, X_val_norm)
        
        general_fold, by_class_fold = compute_accuracies(y_pred, y_val)
        general.append(general_fold)
        by_class.append(by_class_fold)
        
    by_class = np.mean(by_class, axis = 0)

    return general, by_class




## item a)

#### Regressão logística: 

In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def pesosLogistica(X, y, alfa, n_iters):
    #X e um vetor de entradas com a primeira coluna somente com numeros 1's
    n_samples, n_features = X.shape

    w = np.zeros((n_features,1))

    for _ in range(n_iters):
        pred_linear = np.dot(X, w)
        y_pred = sigmoid(pred_linear)
        
        erros = y - y_pred

        w += (alfa/n_samples)*X.T.dot(erros)

    return w

def predicaoLogistica(X_treino, y_treino, X_teste):
    w = pesosLogistica(X_treino,y_treino,0.1,1000)
    probs = sigmoid(np.dot(X_teste, w))
    return (probs >= 0.5).astype(int).ravel() # Regressao logistica binaria


#### Análise do discriminante Gaussiano

In [6]:
def treinarGDA(X_treino, y_treino):
    #y_treino = y_treino.ravel()
    classes = np.unique(y_treino)
    K = len(classes)
    m, n = X_treino.shape

    mu = np.zeros((K, n))
    phi = np.zeros(K)
    sigma = np.zeros((n, n))

    for k in range(K):
        X_k = X_treino[y_treino.ravel() == k]
        mu[k] = X_k.mean(axis=0)
        phi[k] = len(X_k) / m
        sigma += (X_k - mu[k]).T @ (X_k - mu[k])

    sigma /= m
    sigma_inv = np.linalg.inv(sigma)

    return mu, sigma_inv, phi

def predicaoGDA(X_treino, y_treino, X_val):
    mu, sigma_inv, phi = treinarGDA(X_treino, y_treino)
    K = mu.shape[0]
    preds = []

    for x in X_val:
        log_probs = []
        for k in range(K):
            delta = x - mu[k]
            score = -0.5 * delta @ sigma_inv @ delta.T + np.log(phi[k])
            log_probs.append(score)
        preds.append(np.argmax(log_probs))


    return np.array(preds)

#### Naive-Bayes Gaussiano

In [7]:
def treinarNBG(X_treino, y_treino):
    
    #y_treino = y_treino.ravel()
    classes = np.unique(y_treino)
    n_classes = len(classes)
    m, n = X_treino.shape

    priors = np.zeros(n_classes)
    means = np.zeros((n_classes,n))
    variances = np.zeros((n_classes,n))

    for idx, c in enumerate(classes):
        X_c = X_treino[y_treino.ravel() == c]
        priors[idx] = X_c.shape[0] / m
        means[idx, :] = np.mean(X_c, axis = 0)
        variances[idx, :] = np.var(X_c, axis = 0,ddof=1)  

    return classes, priors, means, variances


def previsaoNBG(X_treino, y_treino, X_val):
    classes, priors, means, variances = treinarNBG(X_treino, y_treino)

    predictions = []

    for x in X_val:
        posteriors = []
        for idx, c in enumerate(classes):
            log_likelihood = -0.5 * np.sum(np.log(2 * np.pi * variances[idx] + 1e-9) + ((x - means[idx])**2) / (variances[idx] ))
            posterior = np.log(priors[idx]) + log_likelihood
            posteriors.append(posterior)

        predictions.append(classes[np.argmax(posteriors)])

    return np.array(predictions)




## item b)

In [8]:
bc_data_X1 = np.c_[np.ones(bc_data_X.shape[0]), bc_data_X]

In [18]:
acc, class_acc = k_fold_cross_validation(bc_data_X1,bc_data_y, predicaoLogistica)
print('Acurácia média total para regressão logística: ', np.mean(acc)) 
print('Acurácia média por classe para regressão logística: ', class_acc)



Acurácia média total para regressão logística:  0.9734615384615385
Acurácia média por classe para regressão logística:  [0.97993971 0.9628805 ]


In [27]:
acc, class_acc = k_fold_cross_validation(bc_data_X,bc_data_y, predicaoGDA)
print('Acurácia média total para GDA: ', np.mean(acc)) 
print('Acurácia média por classe para GDA: ', class_acc)


Acurácia média total para GDA:  0.9617032967032966
Acurácia média por classe para GDA:  [0.99772727 0.90331557]


In [33]:
acc, class_acc = k_fold_cross_validation(bc_data_X,bc_data_y, previsaoNBG)
print('Acurácia média total para NBG: ', np.mean(acc)) 
print('Acurácia média por classe para NBG: ', class_acc)

Acurácia média total para NBG:  0.9333791208791208
Acurácia média por classe para NBG:  [0.95579282 0.89243122]


# Questão 2

In [34]:
vehicle = genfromtxt('vehicle.csv', delimiter=',')
vehicle_y = vehicle[:, [18]]
vehicle_X = vehicle[:, :18]

#### One-hot encoding

In [35]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
vehicle_y_encoded = encoder.fit_transform(vehicle_y)
vehicle_y_encoded

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

## item a)

#### Regressão Softmax

In [36]:
def softmax(z):
    exp_z = np.exp(z)
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def treinarsoftmax(X, y, lr=0.1, n_iters=1000):
    
    m, n = X.shape    
    K = y.shape[1]  
    W = np.zeros((n, K))        

    for _ in range(n_iters):
        logits = np.dot(X, W)         
        probs = softmax(logits)       
        W -= lr * (1 / m) * np.dot(X.T, (probs - y))

    return W

def predicaoSoftmax(X_train, y_train, X_test):
    W = treinarsoftmax(X_train, y_train)
    probs = softmax(np.dot(X_test, W))
    return np.argmax(probs, axis=1)

## item b)

In [37]:
vehicle_X1 = np.c_[np.ones(vehicle_X.shape[0]), vehicle_X]

In [42]:
acc, class_acc = k_fold_cross_validation(vehicle_X1, vehicle_y_encoded, predicaoSoftmax)
print('Acurácia média total para regressão softmax: ', np.mean(acc)) 
print('Acurácia média por classe para regressão softmax: ', class_acc)

Acurácia média total para regressão softmax:  0.7526190476190476
Acurácia média por classe para regressão softmax:  [0.91554418 0.55715866 0.56671841 0.97477472]


In [50]:
acc, class_acc = k_fold_cross_validation(vehicle_X, vehicle_y, predicaoGDA)
print('Acurácia média total para GDA: ', np.mean(acc)) 
print('Acurácia média por classe para GDA: ', class_acc)

Acurácia média total para GDA:  0.7763492063492062
Acurácia média por classe para GDA:  [0.95651148 0.6017922  0.60269186 0.94719833]


In [61]:
acc, class_acc = k_fold_cross_validation(vehicle_X, vehicle_y, previsaoNBG)
print('Acurácia média total para NBG: ', np.mean(acc)) 
print('Acurácia média por classe para NBG: ', class_acc)

Acurácia média total para NBG:  0.4660317460317461
Acurácia média por classe para NBG:  [0.17585686 0.42684504 0.40817517 0.88589583]


## Considerações finais: 

Os modelos funcionam consideravelmente melhor para os casos de classificação binária da questão 1, a maior disparidade de acurácia se dá no modelo naive-bayes que performa muito pior no caso multiclasse da questão 2, oque, em tese, é coerente, mas pode ter sido alguma falha na implementação do modelo que não identifiquei. 

Outro ponto importante é que o modelo de regressão logística aplicado no item a) da questão 1 está um pouco duvidoso, devido a uma acurácia muito grande em varios folds. O modelo performa, em média 97-98% de acurácia e chega a ter, em várias iterações dos k-folds, precisão de 100%, o que, muito provavelmente, indica um erro na implementação que não consegui encontrar, visto que, comparando com a implementação de regressão logística e dos k-fold do scikit-learn a precisão média geral não costuma passar 95% para o mesmo dataset. 